# Stocker les données avec Elastic Search

## Imports

In [4]:
import pandas as pd	
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
from faker import Faker
from elasticsearch import Elasticsearch
from sklearn.calibration import CalibratedClassifierCV
import requests

## 1. Mise en place

Après avoir installé l'extension Docker dans VS Code et s'être connecté, on suit les étapes suivantes

a) Démarrer un container à partir de l’image docker.elastic.co/elasticsearch/elasticsearch:7.17.10  
- en mode détaché -d
- monter le volume /usr/share/elasticsearch/data en local
- utiliser le mapping de port -p 9200:9200
- utiliser la variable -e "discovery.type=single-node"
- le nom du container sera --name elastic

commande :   dans le répertoire de travail   
`docker run -d -p 9200:9200 -e "discovery.type=single-node" -v $(pwd):/usr/share/elasticsearch/data --name elastic docker.elastic.co/elasticsearch/elasticsearch:7.17.10`

docker run -d -p 9200:9200 -e "discovery.type=single-node" -v /home/apprenant/Documents/DevIA/Projet_DevIA/NLP_Project:/usr/share/elasticsearch/data --name elastic docker.elastic.co/elasticsearch/elasticsearch:7.17.10


b) Visualiser les logs du container  

commande :   
`docker logs elastic`
 
c) appeler la route racine “/”   

commande :   
`curl http://localhost:9200/`

## 2. Mapper et importer les données

### 2.1 Poster le mapping d’un index nommé “notes”

Il doit contenir :  

- un champ “patient_lastname” qui est un term
- un champ “patient_firstname” qui est un term
- un champ “text”, texte analysé en standard
- un champ “date”, qui est une date
- un champ “patient_left”, qui est un booléen
- un champ “emotion”, qui est un term
- un champ ‘confidence” qui est un float  

Pour poster le mapping de l'index "notes" avec les champs spécifiés dans Elasticsearch, on utilise l'API REST d'Elasticsearch avec cURL :  

Le conteneur Elasticsearch crée à l'étape précédente doit être en cours d'exécution.  

Dans un terminal, il faut entrer la commande suivante pour poster le mapping de l'index "notes" :  

```curl -X PUT "http://localhost:9200/notes" -H 'Content-Type: application/json' -d '
{
  "mappings": {
    "properties": {
      "patient_lastname": {
        "type": "keyword"
      },
      "patient_firstname": {
        "type": "keyword"
      },
      "text": {
        "type": "text",
        "analyzer": "standard"
      },
      "date": {
        "type": "date"
      },
      "patient_left": {
        "type": "boolean"
      },
      "emotion": {
        "type": "keyword"
      },
      "confidence": {
        "type": "float"
      }
    }
  }
}'
```

Cette commande envoie une requête PUT à l'URL http://localhost:9200/notes pour créer l'index "notes" avec le mapping spécifié. Chaque champ est défini avec son type approprié.

### 2.2 Alimenter l’index “notes” à l’aide du jeu de données et de la librairie Faker. 

In [3]:
# # Créer une instance de Faker
# fake = Faker()

# # Créer une instance d'Elasticsearch
# es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# # Générer des données et les indexer dans l'index "notes"
# for _ in range(100):
#     note = {
#         "patient_lastname": fake.last_name(),
#         "patient_firstname": fake.first_name(),
#         "text": fake.text(),
#         "date": fake.date_time_this_decade(),
#         "patient_left": fake.boolean(),
#         "emotion": fake.word(),
#         "confidence": fake.pyfloat(left_digits=2, right_digits=2)
#     }
#     es.index(index="notes", document=note)


Ce code utilise la librairie Faker pour générer des valeurs aléatoires pour chaque champ, puis il utilise le client Elasticsearch pour indexer les données dans l'index "notes". Vous pouvez ajuster le nombre de documents générés en modifiant la valeur de range(100) selon vos besoins.

Les notes générées sont accessibles à l'URL suivante : http://localhost:9200/notes/_search

Exemple de "fakes" notes générées : 

In [13]:
# Créer une instance d'Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Effectuer une requête pour récupérer les notes
result = es.search(index="notes", size=3)  # Récupère les 3 premiers documents, ajustez la taille selon vos besoins

# Parcourir les résultats et afficher les notes
for hit in result['hits']['hits']:
    note = hit['_source']
    print("Patient Last Name:", note['patient_lastname'])
    print("Patient First Name:", note['patient_firstname'])
    print("Text:", note['text'])
    print("Date:", note['date'])
    print("Patient Left:", note['patient_left'])
    print("Emotion:", note['emotion'])
    print("Confidence:", note['confidence'])
    print("-------------------------")

### 2.3 Mettez en place un pipeline utilisant le modèle TF-IDF que vous avez développé avec scikit-learn pour remplir les champs “emotion” et “confidence”

La classe **TextProcessor** est définie dans *preprocessing.py*, puis le pipeline est mis en place dans le fichier *tf-idf.py*, on obient alors le fichier pickle *nlp-pipeline.pkl* que l'on va importer et utiliser pour remplir les champs "emotion" et "confidence".

On remplis les champs de l'index comme suit :   
“patient_lastname”: Faker  
“patient_firstname”: Faker  
text”: CSV  
“date”: Faker  
“patient_left”: Faker  
“emotion”: Model  
‘confidence”: Model  


In [12]:
from elasticsearch import Elasticsearch
from faker import Faker
import csv

# Connexion à Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200,'scheme': 'http'}])

# Instanciation de Faker
fake = Faker()

# Fake patient

patient_list= []
# Generate a tuple of (first_name, last_name)

for i in range(50):
    patient_list.append( (fake.first_name(), fake.last_name()) )

# Chemin vers le fichier CSV
csv_file = '../analyse/data/Emotion_final.csv'

# Nom de l'index Elasticsearch
index_name = 'notes'
truc = 1
# Lecture du fichier CSV et indexation des données
with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Génération des valeurs Faker pour les champs nom et prenom
        row['nom'] = fake.last_name()
        row['prenom'] = fake.first_name()
        if truc ==1:
            print(row)
            truc +=1

        # # Indexation des données dans Elasticsearch
        es.index(index=index_name, body=row)

print("Indexation terminée.")

{'Text': 'i didnt feel humiliated', 'Emotion': 'sadness', 'nom': 'Simpson', 'prenom': 'Jennifer'}


ConnectionTimeout: Connection timed out

## 3. Requêtes

1. En recherchant dans la base elastic search, aboutissez à un data frame permettant d’afficher la répartition des sentiments des textes pour un patient (nom/prénom)

In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd

# Créer une instance d'Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Paramètres du patient
patient_lastname = "Doe"
patient_firstname = "John"

# Requête Elasticsearch pour obtenir la répartition des sentiments pour le patient
query = {
  "query": {
    "bool": {
      "must": [
        {"match": {"patient_lastname": patient_lastname}},
        {"match": {"patient_firstname": patient_firstname}}
      ]
    }
  },
  "aggs": {
    "sentiment_distribution": {
      "terms": {
        "field": "emotion.keyword"
      }
    }
  }
}

# Effectuer la requête
result = es.search(index="notes", body=query, size=0)

# Récupérer les agrégations
aggregations = result['aggregations']['sentiment_distribution']['buckets']

# Créer le dataframe à partir des résultats
df = pd.DataFrame(aggregations, columns=['Sentiment', 'Count'])

# Afficher le dataframe
print(df)


Empty DataFrame
Columns: [Sentiment, Count]
Index: []


/tmp/ipykernel_24072/4203410355.py:31: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="notes", body=query, size=0)
/tmp/ipykernel_24072/4203410355.py:31: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index="notes", body=query, size=0)


2. Élaborez une matrice de sentiments contradictoire (toujours en utilisant la base elastic search.    

On veut savoir parmi les documents classifiés comme *happy*, quel pourcentage contient le mot *sadness*. Puis quel pourcentage contient *fear*, etc.  Et cela, pour tous les sentiments.  
On représente les résultats dans une HeatMap  

3. Pour chacune des étapes du deuil (denial, anger, bargaining, depression, and acceptance) rechercher le nombre de text correspondants à l’aide:  
- d’une recherche pleine  
- d’une fuzzy recherche.  

4. Rechercher les textes :  
- qui doivent matcher l’expression “good day” (must)  
- chez les patients encore en consultation (filter)  
- qui contiennent si possible “to rest” (should)  
- qui ne doivent pas avoir seuil de confiance inférieur à 0.5 s’il existe.   
- Observer la répartition de ces résultats par sentiment  

## Effacer la BDD

In [5]:
# index = 'notes'  # Remplacez par le nom de votre index

# # URL de l'API de suppression des documents
# url = f'http://localhost:9200/{index}/_delete_by_query'

# # Corps de la requête de suppression
# query = {
#     "query": {
#         "match_all": {}
#     }
# }

# # Envoyer la requête de suppression
# response = requests.post(url, json=query)

# # Vérifier la réponse
# if response.status_code == 200:
#     print("Les documents ont été supprimés avec succès.")
# else:
#     print("Une erreur s'est produite lors de la suppression des documents.")
#     print(response.text)

Une erreur s'est produite lors de la suppression des documents.
{"took":114,"timed_out":false,"total":33332,"deleted":0,"batches":1,"version_conflicts":0,"noops":0,"retries":{"bulk":0,"search":0},"throttled_millis":0,"requests_per_second":-1.0,"throttled_until_millis":0,"failures":[{"index":"notes","type":"_doc","id":"5WkoIYkBD5tN_c2p5ra4","cause":{"type":"no_such_file_exception","reason":"/usr/share/elasticsearch/data/nodes/0/indices/xAHyFblnRIibShVJs83lwQ/0/index/write.lock"},"status":500},{"index":"notes","type":"_doc","id":"5mkoIYkBD5tN_c2p5rbg","cause":{"type":"no_such_file_exception","reason":"/usr/share/elasticsearch/data/nodes/0/indices/xAHyFblnRIibShVJs83lwQ/0/index/write.lock"},"status":500},{"index":"notes","type":"_doc","id":"52koIYkBD5tN_c2p57YA","cause":{"type":"no_such_file_exception","reason":"/usr/share/elasticsearch/data/nodes/0/indices/xAHyFblnRIibShVJs83lwQ/0/index/write.lock"},"status":500},{"index":"notes","type":"_doc","id":"6GkoIYkBD5tN_c2p57Yj","cause":{"type":

## Effacer l'index "notes"

In [10]:
# from elasticsearch import Elasticsearch

# # Connexion à Elasticsearch
# es = Elasticsearch(hosts=['http://localhost:9200'])

# # Suppression de l'index "notes"
# index_name = "notes"
# response = es.indices.delete(index=index_name)

# # Vérification de la réponse
# if response['acknowledged']:
#     print(f"L'index '{index_name}' a été supprimé avec succès.")
# else:
#     print(f"Erreur lors de la suppression de l'index '{index_name}'.")

ConnectionTimeout: Connection timed out